# Step 0. Parameter Setting 

In [ ]:
import dtnmonitor_v2 as dtn
import os,sys
import getpass



### P3. NUTTCP THREADS
## used in Step 2 :input threads number
count = 8

### P4. SELECT OUTPUT FILE , 
### used in Step 3 : nuttcp write file , the file number decides the amount of network flow threads 
file_write_nuttcp=[
"/data/disk0/sc17/w11",
"/data/disk1/sc17/w12",
"/data/disk2/sc17/w13",
"/data/disk3/sc17/w14",
"/data/disk4/sc17/w15",
"/data/disk5/sc17/w16",
"/data/disk6/sc17/w17",
"/data/disk7/sc17/w18",
    # "/foo/bar/filennn"
     ]

file2_write_nuttcp=[
"/data/disk0/sc17/w31",
"/data/disk1/sc17/w32",
"/data/disk2/sc17/w33",
"/data/disk3/sc17/w34",
"/data/disk4/sc17/w35",
"/data/disk5/sc17/w36",
"/data/disk6/sc17/w37",
"/data/disk7/sc17/w38",
    # "/foo/bar/filennn"
     ]


### P5.  SELECT THE MONITOR NETWORK INTERFACE  ###
## used in Step 4 , if you don't know the interface name, you can use 'all'
## make sure the interface is correct, error_naming or without network flow will get empty grath
#dtn.interface = 'eth0.1301'
#dtn.interface = 'eth0.2038' 
dtn.interface = 'ens1f0.3061'
#dtn.interface = 'eth1'
#dtn.interface = 'all'



## run fio and check status
### select graph_mode [0|1] 
#graph_mode=0  # all, 
#graph_mode=1  # 100pt
graph_mode=0  

### Show Disk and Interface
import netifaces
import os
list=netifaces.interfaces()
print("==========================================================")
print('{:40s} {:20s} '.format("Network Interface","IP"))
print("==========================================================")
for inf in list:
    #print(inf)
    ff=netifaces.ifaddresses(inf)
    #print(ff)
    try:
        addr=ff[netifaces.AF_INET][0]['addr']
    except:
        continue
    print('{:40s} {:20s} '.format(inf,addr)) 

import getpass
user=getpass.getuser()
import subprocess
import re
df = subprocess.Popen(["df","-h"], stdout=subprocess.PIPE)
output = df.communicate()[0]
dfs = output.decode("utf8").split("\n")
dfs.pop(0)
print("==========================================================")
print('{:35s} {:7s}  {:7s} {:7s}'.\
      format("Disk mountpoint","size","avail","used%"))
print("==========================================================")
for i in dfs:
    try:
        xx = re.sub(" +"," ",i).split(" ")
        if len(xx) == 6:
            device, size, used, available, percent, mountpoint = xx
            print('{:35s} {:7s}  {:7s} {:7s}'\
                  .format(mountpoint,size,available,percent))
    except:
        continue
        
def exec_print(command):
    process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
    for line in iter(process.stdout.readline, b''):
        # system print
        sys.stdout.write(line)      

In [ ]:

print("==========================================================")
print('{:60s} '.format("Initial checking"))
print("==========================================================")

command="ps aux |grep 'python3 '|wc -l"
process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
ret=process.stdout.read().decode('utf8')



if int(ret) > 2: 
    print("System check ..... [ok]")
    print("Error!! Some process are running, please check first!!!")
    exec_print("ps -u "+ user +" ux |grep 'python3 '")
else : 
    print("System check ..... [ok]")
    print('Monitoring Network Interface => {:15s} '.format(dtn.interface))

# Step 1. Start servers for network transfer by disk

In [ ]:


count= len(file_write_nuttcp)

for i in range(count):
    command = "nuttcp -S -1 -sdz -P 5300"+str(i)+" -p 5310"+str(i)+ " > "+ file_write_nuttcp[i] +" & "
    #print(command)
    os.system(command)
    command = "nuttcp -S -1 -sdz -P 5320"+str(i)+" -p 5330"+str(i)+ " > "+ file2_write_nuttcp[i] +" & "
    #print(command)
    os.system(command)
    #dtn.exec_sysout(command)



In [ ]:
exec_print("sleep 3; ps -u "+ user +" ux |grep nuttcp")

# Step 2. Start Monitor

In [ ]:
dtn.exec_command("python callmonitor_v2.py",graph_mode)

## Step A . stop the test nuttcp

In [ ]:
exec_print("pkill nuttcp")

In [ ]:
exec_print("sleep 3; ps -u "+ user +" ux |grep nuttcp")